In [2]:
import warnings
warnings.simplefilter('ignore')

import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.base import BaseEstimator

%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Описание задачи
Построение рекомендательной системы на основе рейтингов, проставленных пользователями подпискам на журналы. Исходные данные взяты с сайта: https://nijianmo.github.io/amazon/index.html

Загрузим датасет и ознакомимся с ним.

In [3]:
data = pd.read_csv('Magazine_Subscriptions.csv', sep=',', decimal='.', header=None, encoding='utf-8')
data.head()

,0,1,2,3
0,B00005N7P0,AH2IFH762VY5U,5.0,1005177600
1,B00005N7P0,AOSFI0JEYU4XM,5.0,1004486400
2,B00005N7OJ,A3JPFWKS83R49V,3.0,1174694400
3,B00005N7OJ,A19FKU6JZQ2ECJ,5.0,1163116800
4,B00005N7P0,A25MDGOMZ2GALN,5.0,1405296000


Согласно описанию, указанному на сайте в первом столбце содержится данные о пользователях, во втором - о товарах, в третьем - о рейтинге, в четвертом - дата в формате timestamp.
Переименуем поля.

In [4]:
data = data.rename({
    0 : 'userId',
    1 : 'itemId',
    2 : 'rating',
    3 : 'timestamp'
                   }, axis=1)
data.head()

,userId,itemId,rating,timestamp
0,B00005N7P0,AH2IFH762VY5U,5.0,1005177600
1,B00005N7P0,AOSFI0JEYU4XM,5.0,1004486400
2,B00005N7OJ,A3JPFWKS83R49V,3.0,1174694400
3,B00005N7OJ,A19FKU6JZQ2ECJ,5.0,1163116800
4,B00005N7P0,A25MDGOMZ2GALN,5.0,1405296000


Возьмем из нашего датасета только половину данных для ускорения работы моделей.

In [5]:
data = data.sample(frac=0.5, random_state=42)
len(data)

44844

## EDA
Проведем базовый EDA.

Посмотрим на распределение рейтинга.

In [6]:
data['rating'].value_counts(normalize=True, dropna=False)

5.0    0.601507
4.0    0.138123
1.0    0.123695
3.0    0.077915
2.0    0.058759
Name: rating, dtype: float64

Из распределения видно, что пользователи чаще ставили оценку 5 (в более половине случаев). Оценку 4 ставили в 14% случаев. Наихудшую оценку 1 ставили в 12% случаев. Оценки 2 и 3 ставили в менее 10% случаев.

Теперь посмотрим сколько уникальных пользователей и товаров присутствуют у нас в базе.

In [7]:
print('Количество уникальных пользователей в данных: ' + str(len(data['userId'].unique())))
print('Количество уникальных товаров в данных: ' + str(len(data['itemId'].unique())))

Количество уникальных пользователей в данных: 2101
Количество уникальных товаров в данных: 39331


## Построение рекомендательной системы
### Разбиение на обучающую и тестовые выборки
Для начала разобъем наши данные на train/test выборки. В качестве метрики оценки качества нашей модели будет использоваться метрика RMSE (средняя квадратичная ошибка).
Запишем расчет ошибки и функцию для разбиения наших данных. Так как в наших данных присутсвует дата, необходимо отсортировать данные по дате, чтобы в тестовую выборку не попали данные из будущего.

In [8]:
rmse = lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred))

def train_test_split(X, ratio=0.2, user_col='userId', item_col='itemId',
                     rating_col='rating', time_col='timestamp'):
    # сортируем оценки по времени
    X.sort_values(by=[time_col], inplace=True)
    # список всех юзеров
    userIds = X[user_col].unique()
    X_train_data = []
    X_test_data = []
    y_train = []
    y_test = []
    for userId in tqdm_notebook(userIds):
        curUser = X[X[user_col] == userId]
        # определяем позицию, по которой делим выборку и размещаем данные по массивам
        idx = int(curUser.shape[0] * (1 - ratio))
        X_train_data.append(curUser[[user_col, item_col]].iloc[:idx, :].values)
        X_test_data.append(curUser[[user_col, item_col]].iloc[idx:, :].values)
        y_train.append(curUser[rating_col].values[:idx])
        y_test.append(curUser[rating_col].values[idx:])
    # cтекуем данные по каждому пользователю в общие массивы
    X_train = pd.DataFrame(np.vstack(X_train_data), columns=[user_col, item_col])
    X_test = pd.DataFrame(np.vstack(X_test_data), columns=[user_col, item_col])
    y_train = np.hstack(y_train)
    y_test = np.hstack(y_test)
    return X_train, X_test, y_train, y_test

In [9]:
X_train, X_test, y_train, y_test = train_test_split(data)

### User-Based Model
Теперь построим модель коллоборативной фильтрации user-based model, основанной на подходе, что похожим пользователям нравятся похожие товары. Запишем класс, в котором определим все аттрибуты нашей модели.

In [10]:
class UserBased(BaseEstimator):
    def fit(self, X, y, user_col='userId', item_col='itemId'):
        X = X.copy()
        # сохраним текущих пользователей и имеющиеся предметы
        self.users = X[user_col].unique()
        self.items = X[item_col].unique()
        
        X['y'] = y
        # рассчитаем среднее значение рейтинга для пользователя и товара
        self.mean_y_user = X.groupby(user_col)['y'].mean()
        self.mean_y_item = X.groupby(item_col)['y'].mean()
        
        # вычитаем среднюю оценку пользователя
        X['y'] -= X[user_col].apply(lambda x: self.mean_y_user[x])
        
        # создаём векторы для каждого пользователя из купленных товаров
        # для неизвестных товаров ставим оценку 0
        self.user_ratings = pd.pivot_table(X, values='y', index=user_col,
                                           columns=item_col, fill_value=0)
        
        # считаем попарную схожесть между юзерами
        self.user_sim = cosine_similarity(self.user_ratings)
        
        # также сделаем словарь - {значение user_col: index в user_ratings}
        self.user_pos = dict()
        for user in self.users:
            self.user_pos[user] = np.argwhere(self.user_ratings.index.values == user)[0][0]
        return self
    
    def predict_rating(self, pr_user, pr_item):
        # если в обучающей выборке нет такого предмета
        # или пользователя, то вернём 0
        if not pr_item in self.items or not pr_user in self.users:
            return 0
        
        # считаем числитель и знаменатель дроби из формулы предсказания
        numerator = self.user_sim[self.user_pos[pr_user]].dot(
                        self.user_ratings.loc[:, pr_item])   
        # вычитаем 1, так как схожесть пользователя с самим собой равна 1,
        # но модель не должна это учитывать
        denominator = np.abs(self.user_sim[self.user_pos[pr_user]]).sum() - 1
        
        return self.mean_y_user[pr_user] + numerator / denominator
    
    def predict(self, X, user_col='userId', item_col='itemId'):
        y = X[[user_col, item_col]].apply(lambda row: self.predict_rating(row[0], row[1]), axis=1)
        return y

Построим модель обучающей выборке

In [11]:
%%time
ub = UserBased().fit(X_train, y_train)

Wall time: 8.95 s


### Item-Based Model
Построим модель колоборативной фильтрации Item-Based Model, основанную на поиске похожих друг на друга товаров.
Запишем класс, в котором определим все атрибуты нашей модели.

In [14]:
class ItemBased(BaseEstimator):
    def fit(self, X, y, user_col='userId', item_col='itemId'):
        X = X.copy()
        # сохраним текущих пользователей и имеющиеся предметы
        self.users = X[user_col].unique()
        self.items = X[item_col].unique()
        
        X['y'] = y
        # рассчитаем среднее значение рейтинга для пользователя и предмета
        self.mean_y_user = X.groupby(user_col)['y'].mean()
        self.mean_y_item = X.groupby(item_col)['y'].mean()
        
        # вычитаем среднюю оценку предмета
        X['y'] -= X[item_col].apply(lambda x: self.mean_y_item[x])
        
        # создаём векторы для каждого фильма с оценками пользователя
        # если пользователь не поставил оценку, то ставим 0
        self.item_ratings = pd.pivot_table(X, values='y', index=item_col,
                                           columns=user_col, fill_value=0)
        
        # считаем попарную схожесть между фильмами
        self.item_sim = cosine_similarity(self.item_ratings)
        
        # также сделаем словарь {значение item_col: index в item_ratings}
        self.item_pos = dict()
        for item in self.items:
            self.item_pos[item] = np.argwhere(self.item_ratings.index.values == item)[0][0]
        return self
    
    def predict_rating(self, pr_user, pr_item):
        # если в обучающей выборке нет такого предмета
        # или пользователя, то вернём 0
        if not pr_item in self.items or not pr_user in self.users:
            return 0
        
        # считаем числитель и знаменатель дроби из формулы предсказания
        numerator = self.item_sim[self.item_pos[pr_item]].dot(
                        self.item_ratings.loc[:, pr_user])   
        # вычитаем 1, так как схожесть предмета с самим собой равна 1,
        # но модель не должна это учитывать
        denominator = np.abs(self.item_sim[self.item_pos[pr_item]]).sum() - 1
        
        return self.mean_y_item[pr_item] + numerator / denominator
    
    def predict(self, X, user_col='userId', item_col='itemId'):
        y = X[[user_col, item_col]].apply(lambda row: self.predict_rating(row[0], row[1]), axis=1)
        return y

Построим модель на обучающей выборке

In [16]:
%%time
ib = ItemBased().fit(X_train, y_train)

Wall time: 32.1 s


### Сравнение двух моделей
Рассчитаем ошибку по двум моделям

In [20]:
rmse_ub = rmse(y_test, ub.predict(X_test).fillna(0))
rmse_ib = rmse(y_test, ib.predict(X_test))
print('RMSE User-Based Model = ' + str(round(rmse_ub, 2)))
print('RMSE Item-Based Model = ' + str(round(rmse_ib, 2)))

RMSE User-Based Model = 4.08
RMSE Item-Based Model = 4.05


По результатам построенных моделей можем сделать вывод, что оба подхода в целом дают одинаковую ошибку при прогнозе.